# load data

In [9]:
!wget https://www.dropbox.com/s/1mn9u50fipbn5oq/foot.zip
!unzip foot.zip

In [10]:
!rm papers.zip
!rm -rf label_images images
!wget https://www.dropbox.com/s/3e8eld5dm9oz908/papers_and_foots.zip
!unzip papers_and_foots.zip


In [11]:
# Install required libs
!pip install -U segmentation-models-pytorch --user 
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

In [12]:
!pip install opencv-python-headless==4.5.2.52

In [13]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import cv2
import matplotlib.pyplot as plt

In [14]:
# get dir
x_train_dir = 'foot/train'
y_train_dir = 'foot/label'

x_valid_dir = 'images'
y_valid_dir = 'label_images'

x_test_dir = 'images'
y_test_dir = 'label_images'


In [15]:
# tạo hàm hỗ trợ việc ta trực quan hoá dữ liệu(see image)

def visualize(**images):
  n = len(images)
  plt.figure(figsize = (16,5))
  for i,(name,img) in enumerate(images.items()):
    plt.subplot(1,n,i+1)
    plt.xticks([])
    plt.xticks([])
    plt.title(" ".join(name.split("_")).title()) 
    plt.imshow(img)
  plt.show()



#data loader

In [16]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [17]:
class Dataset(BaseDataset):
    """ Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """

    CLASSES = {'paper': 255, 'foot': 136,'bg': 0}

    def __init__(
        self,
        image_dir,
        mask_dir,
        classes = None,
        augmentation = None,
        preprocessing = None
    ):
        # Lấy tất cả các tên file có trong mask_dir
        self.file_image_name = os.listdir(image_dir)
        # if '20416.jepg' in self.file_image_name:
        #       self.file_image_name.remove('20416.jepg')
        
        # Lấy các đường dẫn tới tên file
        # Vì 2 tập nó khác nhau về đuổi ảnh nên ta sẽ phải thêm 1 bươc tách đuôi của tập mask vì ta đang dùng chỉ
        # mục theo image
        self.image_fps = [os.path.join(image_dir,image_name) for image_name in self.file_image_name]
        self.mask_fps = [os.path.join(mask_dir,mask_name[:mask_name.index('.')]+'.png') for mask_name in self.file_image_name]
        # chuyển các str trong classes từ giá trị có trong CLASSES
        self.class_values = [self.CLASSES[cls.lower()] for cls in classes]

        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self,i):
      # get path
      file_path  = self.image_fps[i]
      # đọc file
      image = cv2.imread(file_path)
      # chuyển sang màu RGB
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      # đọc hình ảnh thang đo độ xám 
      mask = cv2.imread(self.mask_fps[i],0)
      # trích xuất các lớp nhất định từ masks(ví dụ cars)
      # Để những chỗ có mask = màu tráng là true
      masks = [(mask == v) for v in self.class_values]
      # Chuyển lại về WxHx1
      mask = np.stack(masks, axis=-1).astype('float')
      
      # Tinh chỉnh hình cảnh cắt,đệm,mờ,...
      if self.augmentation:
          sample = self.augmentation(mask = mask,image = image)
          image,mask = sample['image'],sample['mask']
      
      if self.preprocessing:
          sample = self.preprocessing(image = image,mask=mask)
          image,mask = sample['image'],sample['mask']
      
      return image,mask
    def __len__(self):
        return len(self.file_image_name)

In [18]:
dataset = Dataset(x_train_dir,y_train_dir,classes=['paper','foot','bg'])
image,mask = dataset[0]
visualize(
    image = image,
    mask = mask.squeeze()
)


In [19]:
import albumentations as albu

In [20]:
def get_training_augmentation():
  training_transform = [
                        # xoay ngang ảnh
                        albu.HorizontalFlip(p=0.5),
                        # dịch, nếu là 0.5 tối đa chia về = 0.5 kích thước ban đầu,xoay
                        albu.ShiftScaleRotate(shift_limit=0.1,scale_limit=0.5,rotate_limit=0,border_mode=0,p = 1),
                        # padding if heigh or with less than 320
                        albu.PadIfNeeded(min_height=640,min_width=640,p=1),
                        # Cắt bất kì miễn là đủ 320x320
                        albu.RandomCrop(height=640,width=640,p=1),
                        albu.GaussNoise(p=0.5),
                        albu.Perspective(p=0.5),
                        
                        albu.OneOf(
                            [
                            # Cân bằng biểu đồ thích ứng có giới hạn độ tương phản cho hình ảnh đầu vào
                            albu.CLAHE(p=1),
                            # Chỉnh độ sáng
                            albu.RandomBrightness(p=1),
                            albu.RandomGamma(p=1),
                            ],p=0.9
                        ),

                        albu.OneOf(
                            [
                             # Làm nét ảnh
                             albu.Sharpen(p=1),
                             # Làm mờ ảnh, dùng tối đa đến 3x3
                             albu.Blur(blur_limit=3,p=1),
                              # Làm mờ đối tượng đang chuyển động, dùng tối đa đến 3x3
                             albu.MotionBlur(blur_limit=3,p=1)
                            ],p=0.9
                        ),

                        albu.OneOf([
                               # tạo độ tương phản
                               albu.RandomContrast(p=1),
                               #Dùng để hiệu chỉnh màu sắc, độ bão hòa, giá trị của ảnh
                               albu.HueSaturationValue(p=1),     
                        ],p=0.9
                        )
                ]
  return albu.Compose(training_transform)


def get_validation_augmention():
  validation_transform=[
      albu.PadIfNeeded(min_height= 960,min_width=960)    , 
  ]
  return albu.Compose(validation_transform)
def get_testing_augmention():
  validation_transform=[
      albu.PadIfNeeded(min_height= None,min_width=None,pad_height_divisor=32,pad_width_divisor=32)                 
  ]
  return albu.Compose(validation_transform)
def to_tensor(x, **kwargs):
    # đưa về dạng chuẩn 
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [21]:
dataset = Dataset(x_train_dir,y_train_dir,classes=['paper','foot','bg'],augmentation=get_training_augmentation())
image,mask = dataset[0]
print(image.shape)
# visualize(
#     image = image,
#     mask = mask.squeeze(),
# )


# Create model and training

In [22]:
#cd /root/.local/lib/python3.7/site-packages

In [23]:
import torch
import numpy as np
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils

In [24]:
ENCODER = 'resnet34' # mạng resnet 18
ENCODER_WEIGHTS = 'imagenet' # trọng số được đào tạo trc cho việc khởi tạo encoder là resnet18
CLASSES = ['paper','foot','bg'] 
ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS) 

In [25]:
cd /content

In [26]:
!pip install Image

In [27]:
train_dataset = Dataset(
    image_dir = x_train_dir,
    mask_dir = y_train_dir,
    classes = CLASSES,
    augmentation = get_training_augmentation(),
    preprocessing = get_preprocessing(preprocessing_fn),
)
valid_dataset = Dataset(
    x_valid_dir,
    y_valid_dir,
    CLASSES,
    get_testing_augmention(),
    get_preprocessing(preprocessing_fn),
)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset,batch_size=1, shuffle=True, num_workers=4)

In [28]:
loss = smp.utils.losses.DiceLoss()
metrics = [
           smp.utils.metrics.IoU(threshold=0.5)
]
optimizer = torch.optim.Adam([
                            dict(params = model.parameters(),lr = 0.0001)
])

# Create train epoch and valid epoch

In [29]:
train_epoch = smp.utils.train.TrainEpoch(
    model = model,
    loss = loss,
    metrics = metrics,
    optimizer = optimizer,
    device = DEVICE,
    verbose=True,
)
valid_epoch = smp.utils.train.ValidEpoch(
    model = model,
    loss = loss,
    metrics = metrics,
    device = DEVICE,
    verbose = True,
)


In [30]:
# tao n epoch
max = 0
n  = 50
for i in range(n):
  print("\n Epoch: {}".format(i+1))
  train_logs = train_epoch.run(train_loader)
  valid_logs = valid_epoch.run(valid_loader)
  if max < valid_logs['iou_score'] :
    max = valid_logs['iou_score']
    torch.save(model, './best_model.pth')
    print('Model saved!')
  if i==50:
    optimizer.param_groups[0]['lr'] = 1e-6


# load best model

In [31]:
# load best saved checkpoint
best_model = torch.load('./best_model.pth')

In [32]:
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    augmentation=get_testing_augmention(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)
test_dataloader = DataLoader(test_dataset)


In [33]:
test_epoch = smp.utils.train.ValidEpoch(
    model = best_model,
    loss = loss,
    metrics = metrics,
    device = DEVICE,
   
)

logs = test_epoch.run(test_dataloader)

In [34]:
def SelectImageByArea(pr_mask): # Chọn phần ảnh có kích thước lớn nhất trong hình
  # Đọc phần connected component để biết thêm về dạng kết nối 4 và 8
  pr_mask = np.uint8(pr_mask)
  output = cv2.connectedComponentsWithStats(pr_mask,4,cv2.CV_8S)
  (totalLabels, label_ids, values, centroid) = output
  output = np.zeros(pr_mask.shape, dtype="uint8")
  max = 0
  for i in range(1, totalLabels):
    area = values[i, cv2.CC_STAT_AREA] 
    width = values[i,cv2.CC_STAT_WIDTH]
    height = values[i,cv2.CC_STAT_HEIGHT]
    if area > max:
      width = width
      height = height
      max = area
      componentMask = (label_ids == i).astype('int')
  return componentMask

# visualize predictions

In [35]:
# test_dataset_vis = Dataset(
#     x_test_dir, y_test_dir, 
#     classes=CLASSES,
# )
test_dataset_vis = Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=CLASSES,
)


In [36]:
for n in range(len(test_dataset_vis)):
      n = 5
      image_vis = test_dataset_vis[n][0].astype('uint8')
      image, gt_mask = test_dataset[n]
      gt_mask = gt_mask.squeeze()
      x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
      pr_mask = best_model.predict(x_tensor)
      pr_mask = (pr_mask.squeeze().cpu().numpy().round())
      try:
        visualize(
            image=image_vis, 
            ground_truth_mask=gt_mask.transpose((1,2,0)), 
            predicted_mask=pr_mask.transpose((1,2,0)),
        )
      except:
        print("Error with n = ",n)
      break
  





In [37]:
!pip install imutils

In [38]:
from imutils import perspective

In [39]:
!wget https://www.dropbox.com/s/wjei9164ayfuvqx/data.zip

In [40]:
!unzip data.zip

In [41]:
!wget https://www.dropbox.com/s/fq5brkhi07wa394/test.zip
!unzip test.zip

In [56]:

img = cv2.imread("data/z3523951470241_ed29ef359424772f5378ebdbdc734876.jpg")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
a = get_testing_augmention()
image = a(image=image)['image']
image = get_preprocessing(preprocessing_fn)(image=image)['image']
image = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
pr_mask = best_model.predict(image)
pr_mask = (pr_mask.squeeze().cpu().numpy().round())




In [57]:
def MAX(a,b):
    if a>b:
        return a
    else:
        return b
def MIN(a,b):
    if a<b:
        return a
    else:
        return b

In [58]:
# đo chiều rộng
paper = np.uint8(SelectImageByArea(pr_mask[0]))
img = np.zeros_like(paper)
# Tìm cạnh
contours, hierarchy = cv2.findContours (paper, cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)
# Vẽ đa giác lồi
hull_list = []
for i in contours:
    hull = cv2.convexHull(i)
    hull_list.append(hull)
cv2.drawContours(img, hull_list,-1,20,20)
# Chọn phần có diện tích lớn nhất chống nhiễu
img = np.uint8(SelectImageByArea(img))
visualize(paper = img)
# Tìm 4 góc
corners = np.array(cv2.goodFeaturesToTrack(img,4,0.01,50).astype(int))
corner = []
for i in corners:
    x,y = i.ravel()
    corner.append([x,y])
# Sắp xếp theo thứ tự top_left -> top_right -> bottom_right -> bottom_left
corners = perspective.order_points(np.array(corner)).astype("float32")
corners = np.array(corners)
tl, tr, br, bl = np.array(corners)
# Tìm max heigh,width
widthA = np.linalg.norm(tl-tr)
widthB = np.linalg.norm(bl-br)
maxWidth = MAX(int(widthA), int(widthB))
heightA = np.linalg.norm(tl-bl)
heightB = np.linalg.norm(tr-br)
maxHeight = MAX(int(heightA), int(heightB))
# 1px ~ cm
px = 21/MIN(maxHeight,maxWidth)
# perspective
dst = np.array([
        [0, 0],
        [maxWidth, 0],
        [maxWidth , maxHeight],
        [0, maxHeight]], dtype = "float32")
M = cv2.getPerspectiveTransform(corners, dst)
paper = cv2.warpPerspective(paper, M, (maxWidth, maxHeight))

ori_paper = paper.copy()
paper1 = paper.copy()
paper = np.array((np.array(paper) == 0).astype(int))


x2 = int(paper.shape[0]*0.9)

y2 = int(paper.shape[1]*0.9)
# Tìm cạnh của chân có trong ảnh
contours, hierarchy = cv2.findContours (np.uint8(paper[0:x2,0:y2]), cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)
Max = 0
for i in contours:
    if cv2.contourArea(i) > Max:
        Max = cv2.contourArea(i)
        box = cv2.minAreaRect(i)
        # Lấy tọa độ các đỉnh của MinRect
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
corners = perspective.order_points(np.array(box)).astype("float32")
corners = np.array(corners)
# Vẽ mình hoạ
cv2.drawContours(paper1, [box.astype("int")], -1, (2,0,0), 10)
tl, tr, br, bl = np.array(corners)
width = MIN(np.linalg.norm(tl-bl),np.linalg.norm(bl-br))
print("width: ",width*px)

In [59]:
visualize(ori = pr_mask.transpose(1,2,0),paper1 = paper1,ori_paper = ori_paper)

In [66]:
img = cv2.imread("test/290308056_572147007626053_9171302520216046212_n.jpg")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
a = get_testing_augmention()
image = a(image=image)['image']
image = get_preprocessing(preprocessing_fn)(image=image)['image']
image = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
pr_mask = best_model.predict(image)
pr_mask = (pr_mask.squeeze().cpu().numpy().round())



In [67]:
def SortByY(a):
    return a[1]
def SortByX(a):
    return a[0]

In [68]:
# Đưa mô hình lên web

In [70]:
paper = np.uint8(SelectImageByArea(pr_mask[0]))
img = np.zeros_like(paper)
img1 = np.zeros_like(paper)
# Tìm cạnh
contours, hierarchy = cv2.findContours (paper, cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)
# Nắn các cạnh cong (kiểu vậy)
for c in contours:
    epsilon = 0.03 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, epsilon, True)
    cv2.drawContours(img, [approx], -1, (255, 0, 0), 20)
# Tìm lại cạnh
contours, hierarchy = cv2.findContours (img, cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)
# Tìm và vẽ lồi
hull_list = []
for i in contours:
    hull = cv2.convexHull(i)
    hull_list.append(hull)
cv2.drawContours(img1, hull_list,-1,20,20)
# sau các bước trên ta vẽ da giác lồi đó vào img1
img1 = np.uint8(SelectImageByArea(img1))
# Lấy 5 đỉnh
corners = np.array(cv2.goodFeaturesToTrack(img1,5,0.02,50).astype(int))
corner = []
for i in corners:
    x,y = i.ravel()
    corner.append([x,y])
# Vẽ mình hoạ
for i in corner:
    cv2.circle(img,(i[0],i[1]), 63, (255,255,255), -1)
visualize(img = img)

# Tìm giao điểm của 2 đường thẳng
# 
arr1 = sorted(corner,key=SortByY)
arr2 = sorted(corner,key=SortByX,reverse = True)
# Lấy toạ độ các đỉnh
Y1 = arr1[0]
Y2 = arr1[1]
X1 = arr2[0]
X2 = arr2[1]
# Tính hệ số
a1 = Y2[1] - Y1[1]
b1 = -Y2[0] + Y1[0]
c1 = -a1*Y1[0] - b1*Y1[1]

a2 = X2[1] - X1[1]
b2 = -X2[0] + X1[0]
c2 = -a2*X1[0] - b2*X1[1]
# Tìm đỉnh giao
result = np.linalg.inv(np.array([[a1,b1],[a2,b2]])).dot(np.array([-c1,-c2])).astype(int)
# Lấy 3 đỉnh chính của tờ giấy (không có góc bị khuất)
corners = np.array(cv2.goodFeaturesToTrack(img1,3,0.02,10).astype(int))
print(corners)
corner = []
for i in corners:
    x,y = i.ravel()
    corner.append([x,y])
# Thêm góc bị khuất vào
corner.append(result)
corners = perspective.order_points(np.array(corner)).astype("float32")
corners = np.array(corners)
# Đoạn này tương tự như ở trên phần tính chiều rộng
tl, tr, br, bl = np.array(corners)
widthA = np.linalg.norm(tl-tr)
widthB = np.linalg.norm(bl-br)
maxWidth = MAX(int(widthA), int(widthB))
heightA = np.linalg.norm(tl-bl)
heightB = np.linalg.norm(tr-br)
maxHeight = MAX(int(heightA), int(heightB))
px = 29.7/MAX(maxHeight,maxWidth)
dst = np.array([
        [0, 0],
        [maxWidth, 0],
        [maxWidth , maxHeight],
        [0, maxHeight]], dtype = "float32")
M = cv2.getPerspectiveTransform(corners, dst)
paper = cv2.warpPerspective(paper, M, (maxWidth, maxHeight))
ori_paper = paper.copy()
paper1 = paper.copy()
paper = np.array((np.array(paper) == 0).astype(int))


x2 = int(paper.shape[0]*0.1)

y2 = int(paper.shape[1]*0.95)

contours, hierarchy = cv2.findContours (np.uint8(paper[:,:y2]), cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)
Max = 0
for i in contours:
    if cv2.contourArea(i) > Max:
        Max = cv2.contourArea(i)
        box = cv2.minAreaRect(i)
        # Lấy tọa độ các đỉnh của MinRect
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
corners = perspective.order_points(np.array(box)).astype("float32")
corners = np.array(corners)

cv2.drawContours(paper1, [box.astype("int")], -1, (2,0,0), 10)
tl, tr, br, bl = np.array(corners)
height = MAX(np.linalg.norm(tl-bl),np.linalg.norm(bl-br))
print("Height: ",height*px)

visualize(ori = pr_mask.transpose(1,2,0),paper1 = paper1,ori_paper = ori_paper)

In [50]:
!pip install streamlit
!pip install pyngrok
!pip install Pillow

In [51]:
!wget https://www.dropbox.com/s/zv5kg4ouwcpshrd/main.py
!wget https://www.dropbox.com/s/do4emfwtxiz0okk/myutils.py

In [52]:
from pyngrok import ngrok
!ngrok authtoken 2ARIBeOv16CSwiLMroo2AGNKvpM_6Y9zhbCtJGYBW5qqNXRKR

In [53]:
!rm -r upload

In [54]:
!mkdir upload
!mkdir upload/image upload/mask

In [55]:
!streamlit run --server.port 80 main.py &>/dev/null&
public_url = ngrok.connect(port='8501', proto='http')
public_url

In [ ]:
# !ps aux | grep ngrok

In [ ]:
# !kill -9 1526

In [ ]:
img = cv2.imread("label_images/24042.png")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)
#result is dilated for marking the corners, not important
kernel = np.ones((10,10))
dst = cv2.dilate(dst,kernel)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.6*dst.max()]=[255,0,0]

plt.imshow(img)